In [1]:
import pandas as pd
import numpy as np
import os

In [6]:
df_curr=pd.read_csv("/kaggle/input/ai-transformer-monitoring/CurrentVoltage.csv")
df_over=pd.read_csv("/kaggle/input/ai-transformer-monitoring/Overview.csv")
df_power=pd.read_csv("/kaggle/input/ai-transformer-monitoring/Power.csv")

In [7]:
concaty=pd.concat([df_curr,df_over,df_power],axis=1)

In [13]:
concaty2=concaty.dropna(how='any').drop(columns=["DeviceTimeStamp"])

In [16]:
# concaty["DeviceTimeStamp"]=pd.to_datetime(concaty["DeviceTimeStamp"])

In [14]:
concaty2

,VL1,VL2,VL3,IL1,IL2,IL3,VL12,VL23,VL31,INUT,...,MOG_A,WL1,WL2,WL3,VAL1,VAL2,VAL3,RVAL1,RVAL2,RVAL3
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
1,238.7,238.7,238.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
2,238.4,238.5,238.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
3,239.9,240.0,240.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
4,239.9,240.0,240.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19304,244.6,243.1,242.3,58.7,39.7,82.5,423.0,419.5,422.8,36.6,...,0.0,19.3,10.9,20.5,19.3,11.0,21.0,1.0,1.7,0.0045
19305,244.4,243.0,242.2,63.3,54.6,93.4,422.5,419.2,422.6,35.4,...,0.0,18.8,9.9,20.1,18.9,10.2,20.6,1.3,2.4,0.0044
19306,244.1,242.6,242.1,60.3,41.2,79.2,422.1,418.9,422.3,32.7,...,0.0,19.0,8.9,18.1,19.0,9.1,18.3,0.7,2.0,0.0030
19307,243.7,242.4,241.8,63.4,31.9,74.7,421.7,418.5,421.6,38.8,...,0.0,17.4,8.8,21.4,17.3,8.9,21.6,0.3,1.6,0.0035


# # Deep learning

In [22]:
concaty_reshape=concaty2.values
concaty_reshaped=np.repeat(concaty_reshape[:,np.newaxis,:],15,axis=1)
concaty_reshaped.shape

(19309, 15, 26)

In [24]:
from keras.layers import Dense,LSTM,SimpleRNN,GRU,BatchNormalization,MultiHeadAttention,Input,Dropout,TimeDistributed
from keras.regularizers import l2
from keras.initializers import HeNormal
from keras.callbacks import EarlyStopping
from keras.models import Model

In [29]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [32]:
inputs=Input(shape=(15,26))
e=SimpleRNN(32,activation="relu",return_sequences=True,kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(inputs)
e=LSTM(27,activation="relu",return_sequences=True,kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(e)
e=GRU(22,activation="relu",return_sequences=True,kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(e)

bottleneck=SimpleRNN(15,activation="relu",return_sequences=True,kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(e)

d=GRU(22,activation="relu",return_sequences=True,kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(bottleneck)
d=LSTM(27,activation="relu",return_sequences=True,kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(d)
d=SimpleRNN(32,activation="relu",return_sequences=True,kernel_regularizer=l2(0.01),kernel_initializer=HeNormal())(d)

outputs=TimeDistributed(Dense(26,activation="softmax"))(d)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 15, 26)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_30 (SimpleRNN)       │ (None, 15, 32)         │         1,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 15, 27)         │         6,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_6 (GRU)                     │ (None, 15, 22)         │         3,366 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_31 (SimpleRNN)       │ (None, 15, 15)         │           570 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_7 (GRU)                     │ (None, 15, 22)         │         2,574 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 15, 27)         │         5,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_32 (SimpleRNN)       │ (None, 15, 32)         │         1,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, 15, 26)         │           858 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,056 (90.06 KB)

 Trainable params: 23,056 (90.06 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
model.fit(concaty_reshaped,concaty_reshaped,epochs=32,batch_size=32,validation_split=0.2,callbacks=[early_stopping])

Epoch 1/32
483/483 ━━━━━━━━━━━━━━━━━━━━ 39s 54ms/step - accuracy: 0.0589 - loss: 32248.5762 - val_accuracy: 0.0165 - val_loss: 33187.7461
Epoch 2/32
483/483 ━━━━━━━━━━━━━━━━━━━━ 41s 53ms/step - accuracy: 0.0947 - loss: 30428.8359 - val_accuracy: 0.0166 - val_loss: 33095.0117
Epoch 3/32
483/483 ━━━━━━━━━━━━━━━━━━━━ 25s 51ms/step - accuracy: 0.0948 - loss: 30323.5508 - val_accuracy: 0.0444 - val_loss: 32691.8477
Epoch 4/32
483/483 ━━━━━━━━━━━━━━━━━━━━ 25s 51ms/step - accuracy: 0.1521 - loss: 29081.7441 - val_accuracy: 0.5231 - val_loss: 24475.2812
Epoch 5/32
483/483 ━━━━━━━━━━━━━━━━━━━━ 25s 52ms/step - accuracy: 0.3853 - loss: 22932.5000 - val_accuracy: 0.5335 - val_loss: 24468.1172
Epoch 6/32
483/483 ━━━━━━━━━━━━━━━━━━━━ 24s 50ms/step - accuracy: 0.3684 - loss: 22940.0078 - val_accuracy: 0.5151 - val_loss: 24466.5352
Epoch 7/32
483/483 ━━━━━━━━━━━━━━━━━━━━ 25s 51ms/step - accuracy: 0.3477 - loss: 21336.8047 - val_accuracy: 0.3563 - val_loss: 5869.9229
Epoch 8/32
483/483 ━━━━━━━━━━━━━━━━

In [35]:
prediction=model.predict(concaty_reshaped)
reconstruction_error=np.mean(np.square(concaty_reshaped-prediction))
threshold=np.mean(reconstruction_error)+2*np.std(reconstruction_error)
labels=np.where(reconstruction_error>threshold,0,1)
accuracy=np.mean(labels==0)*100
print(accuracy)

604/604 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step
0.0
